In [11]:
from transformers import DistilBertTokenizer, DistilBertModel
from transformers import CamembertTokenizer, CamembertModel
from transformers import BertTokenizer, BertModel
from transformers import FlaubertTokenizer, FlaubertModel
from transformers import AutoTokenizer, BloomForCausalLM
import torch.nn as nn
import faiss
import ast
import pandas as pd
import torch

class ContrastiveModel(nn.Module):
    def __init__(self, model_name, embedding_dim):
        super(ContrastiveModel, self).__init__()
        self.bert = globals()[f'{model_suffix}Model'].from_pretrained(model_name)
        self.projection = nn.Sequential(
            nn.Linear(self.bert.config.hidden_size, embedding_dim),
            nn.ReLU(),
            nn.Linear(embedding_dim, embedding_dim)
        )

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        embeddings = outputs.last_hidden_state[:, 0] 
        return self.projection(embeddings)
        
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

df_embeddings = pd.read_csv('./finetuned/embedding_matrix.csv')
df_embeddings['Embedding'] = df_embeddings['Embedding'].apply(ast.literal_eval)
texts = df_embeddings['Text'].tolist()
embeddings = torch.tensor(df_embeddings['Embedding'].tolist())


models_names = ['flaubert/flaubert_base_cased', "camembert-base", "bert-base-multilingual-cased", 'distilbert-base-uncased']
models_suffix = ['Flaubert', 'Camembert', 'Bert', 'DistilBert']
model_index = 0
model_name = models_names[model_index]
model_suffix = models_suffix[model_index]



tokenizer_loaded = AutoTokenizer.from_pretrained('./finetuned/model')
model_loaded = ContrastiveModel(model_name=model_name, embedding_dim=embeddings.shape[1])  
model_loaded.load_state_dict(torch.load('./finetuned/contrastive_model.pth', map_location=torch.device('cpu')), strict=False)
model_loaded.eval().to('cuda' if torch.cuda.is_available() else 'cpu')

bloom_tokenizer = AutoTokenizer.from_pretrained("bigscience/bloom-560m") 
bloom_loaded = BloomForCausalLM.from_pretrained("bigscience/bloom-560m")

index = faiss.IndexFlatL2(embeddings.shape[1]) 
index.add(embeddings.numpy()) 

print(f"Index construit avec {index.ntotal} documents.")

Using device: cpu


`do_lowercase_and_remove_accent` is passed as a keyword argument, but this won't do anything. `FlaubertTokenizer` will always set it to `False`.
C:\Users\Nedved\AppData\Local\Temp\ipykernel_16340\1532563067.py:46: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file.

Index construit avec 8264 documents.


In [ ]:
# pip install sacremoses

In [28]:
def embed_text(query, model, tokenizer, device):
    model.eval()
    tokens = tokenizer(query, padding=True, truncation=True, max_length=512, return_tensors="pt").to(device)
    with torch.no_grad():
        output = model(tokens['input_ids'], tokens['attention_mask'])
    embeddings = output[0]
    if len(embeddings.shape) == 1:
        return embeddings  
    else:
        return embeddings.mean(dim=1)  
        
user_query = "Je dois faire des modification dans un établissement recevant le public. Par qui l'autorisation est elle délivrée"
user_query = "J'ai fais des travaux dans un établissement recevant des travailleurs, qu'est ce que je dois faire des déchets de chantier"

def retrieve(query, model, tokenizer, index, texts, top_k=3):
    query_embedding = embed_text(query, model, tokenizer, device).unsqueeze(0)
    distances, indices = index.search(query_embedding.to('cpu').numpy(), top_k)
    fais_results = [{"text": texts[i], "distance": distances[0][j]} for j, i in enumerate(indices[0])]
    print(fais_results)
    return fais_results

def generate_answer(context, query, model, tokenizer):
    input_text = f"Context: {context} Query: {query}"
    input_ids = bloom_tokenizer.encode(input_text, return_tensors='pt', max_length=1024, truncation=True, padding=True).to('cuda' if torch.cuda.is_available() else 'cpu')
    max_new_tokens = 1024 - input_ids.shape[1] +1
    outputs = bloom_loaded.generate(input_ids, max_new_tokens=max_new_tokens, num_beams=5, early_stopping=True)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

def rag_pipeline(user_query, model, tokenizer, index, texts):
    retrieved = retrieve(user_query, model_loaded, tokenizer_loaded, index, texts, top_k=5)
    context = " ".join([item['text'] for item in retrieved])
    response = generate_answer(context, user_query, model, tokenizer)
    return response

response = rag_pipeline(user_query, model_loaded, tokenizer_loaded, index, texts)
print("Réponse générée :\n", response)


[{'text': 'Décors§ 1. Les dispositions de l arrêté  portant réglementation de l utilisation de certains matériaux et  produits dans les établissements recevant du public sont applicables aux  décors concernés.§ 2. Les décors doivent être en matériaux M1 ou classés B-s2, d0.La salle a un désenfumage de classe 2.§  3. Toutefois, les décors en matériaux M2 ou classés C-s2, d0 ou en bois  classés M3 ou D-s2, d0 sont admis si toutes les dispositions suivantes  sont respectées :- le nombre de sorties et le nombre  d unités de passage de la salle sont majorés d un tiers, chaque sortie  ayant une largeur minimale de trois unités de passage ;- une installation de R IA D N 19/6 mm est installée dans la salle ;- la scène est équipée d une extinction automatique d incendie de type déluge ;- la salle a un désenfumage de classe 3 suivant les dispositions définies par l annexe de l I T 246 ;- la distance minimum entre le public et le décor est de 2 mètres ;- l emploi d artifices, de flammes et de bou